In [91]:
import numpy as np
import scipy as sc

from src.Tools import portfolio_metrics
from qiskit_src.ansatz import CP_QAOA, qubo_cost
from qiskit_src.tools import get_qubo, min_cost_partition

In [92]:
N=16
k=8
seed=0
alpha=0.001
layers=3
max_iter=500
w_z_phase = False

In [93]:
expected_returns, covariances = portfolio_metrics(n=N, seed=seed)

constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=N,
                                                            k=k,
                                                            mu=expected_returns,
                                                            sigma=covariances,
                                                            alpha=alpha)

max_cost, min_cost, min_state = constrained_result['c_max'], constrained_result['c_min'], constrained_result['s']
min_state_str = ''.join([str(_) for _ in min_state])
Q, offset = get_qubo(mu=expected_returns,
                     sigma=covariances,
                     alpha=alpha,
                     lmbda=lmbda,
                     k=k)

In [94]:


_available_methods_ = ['COBYLA', 'Nelder-Mead']
_method_idx_ = 0

ansatz = CP_QAOA(N_qubits=N,
                 cardinality=k,
                 layers=layers,
                 QUBO_matrix=Q,
                 QUBO_offset=offset,
                 with_z_phase=w_z_phase)

# Initial guess for parameters (gamma, beta) of circuit
theta_min, theta_max = -np.pi, np.pi
N_xx_yy_angles = layers * (N - 1) 
if w_z_phase:
    N_xx_yy_angles += N * layers
#theta_i = np.random.uniform(low=theta_min, high=theta_max, size=N_xx_yy_angles)
theta_i = np.random.normal(loc=0, scale=1, size=N_xx_yy_angles)
# Use the get_cost method of the specific ansatz instance
res = sc.optimize.minimize(fun=ansatz.get_cost, x0=theta_i,
                           method=_available_methods_[_method_idx_],
                           options={'disp': False, 'maxiter': max_iter})

_dict_ = ansatz.get_state_probabilities(angles=res.x, flip_states=False)
res

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.02388492644409179
       x: [ 3.327e-01 -3.335e-02 ... -5.992e-01  1.355e-01]
    nfev: 489
   maxcv: 0.0

In [95]:
best_state = list(_dict_.keys())[np.argmax(list(_dict_.values()))]
best_state, _dict_[best_state] 

('0101001010111001', 0.11680703559314)

In [96]:
min_cost, max_cost

(-5.791631404929497, -2.7449612462420463)

In [97]:
min_state_str

'0110010110101100'

In [98]:
_dict_[min_state_str] 

KeyError: '0110010110101100'

In [99]:
opt_cost = qubo_cost(min_state.astype(np.float64),Q)+offset
opt_cost

-5.79163140492949

In [100]:
found_cost = qubo_cost(np.array([float(_) for _ in best_state]).astype(np.float64), Q) + offset
found_cost

-4.327302795980131

In [101]:
print(f'Deviation from optimal cost: {round(100*((opt_cost/found_cost) - 1),3)} %')

Deviation from optimal cost: 33.839 %


In [102]:
print(f'Normalized cost: {abs(found_cost - min_cost) / abs(max_cost - min_cost)}')

Normalized cost: 0.48063247174095786
